# Notebook Parameter Calculation

The pourpose of this notebook is to estimate the parameters of a of three-phase induction motor.

In [31]:
import math
import cmath
import yaml

## Book Keeping

- Type: 
- Verbosity:

In [32]:
TYPE = "\"M2BAX 132MA 4 IMB3/IM1001\""
VERBOSITY = 1

## Relevant Test Report Parameters

In [33]:
LINE_RESISTANCE = [0.92008, 0.92008, 0.92047]
FREQUENCY = 50
MOMENT_OF_INERTIA = 0.032
LINE_VOLTAGE_NO_LOAD = 400 # 402.3
INPUT_CURRENT_NO_LOAD = 8.8
POWER_FACTOR_NO_LOAD = 0.06
LINE_VOLTAGE_BLOCKED_ROTOR = 86.5
INPUT_CURRENT_BLOCKED_ROTOR = 15.3
POWER_FACTOR_BLOCKED_ROTOR = 0.40
ROTOR_SPEED_75_PERCENT_LOAD = 1468
SYNCHRONOUS_SPEED = 1500 # 1498
OUTPUT_POWER_75_PERCENT_LOAD = 5.62e3
LINE_VOLTAGE_75_PERCENT_LOAD = 398.9
INPUT_CURRENT_75_PERCENT_LOAD = 13.2
POWER_FACTOR_75_PERCENT_LOAD = 0.70
LINE_VOLTAGE_100_PERCENT_LOAD = 399.3
INPUT_CURRENT_100_PERCENT_LOAD = 16
POWER_FACTOR_100_PERCENT_LOAD = 0.77
NUMBER_OF_POLES = 4

In [34]:
Rs = (sum(LINE_RESISTANCE) / len(LINE_RESISTANCE)) / 2

if VERBOSITY:
    print("Rs :: {:.4f}".format(Rs))

Rs :: 0.4601


In [35]:
Zbr = (LINE_VOLTAGE_BLOCKED_ROTOR / math.sqrt(3)) / INPUT_CURRENT_BLOCKED_ROTOR

Xs = 0.5 * Zbr * math.sin(math.acos(POWER_FACTOR_BLOCKED_ROTOR))
Ls = Xs / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Ls :: {:.4f}".format(Ls * 1000))

Ls :: 4.7613


In [36]:
Vsnl = LINE_VOLTAGE_NO_LOAD / math.sqrt(3)
Isnl = cmath.rect(INPUT_CURRENT_NO_LOAD, -math.acos(POWER_FACTOR_NO_LOAD))
Snl = Vsnl * Isnl.conjugate()
Vmsl = Vsnl - Isnl * complex(Rs, Xs)

Xm = abs(Vmsl)**2 / (Snl.imag - abs(Isnl)**2 * Xs)
Rm = abs(Vmsl)**2 / (Snl.real - abs(Isnl)**2 * Rs)

Lm = Xs / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Rm :: {:.4f}".format(Rm))
    print("Lm :: {:.4f}m".format(Lm * 1000))

Rm :: 548.5375
Lm :: 4.7613m


In [37]:
s75 = (SYNCHRONOUS_SPEED - ROTOR_SPEED_75_PERCENT_LOAD) / SYNCHRONOUS_SPEED

Vs75 = LINE_VOLTAGE_75_PERCENT_LOAD / math.sqrt(3)
Is75 = cmath.rect(INPUT_CURRENT_75_PERCENT_LOAD, -math.acos(POWER_FACTOR_75_PERCENT_LOAD))
S75 = Vs75 * Is75.conjugate()
Vm75 = Vs75 - Is75 * complex(Rs, Xs)
Im75 = Vm75 / (complex(0, Rm * Xm) / complex(Rm, Xm))
Ir75 = Is75 - Im75

Rr = ((S75.real - abs(Vm75)**2 / Rm - abs(Is75)**2 * Rs) * s75) / abs(Ir75)**2
Xr = (S75.imag - abs(Vm75)**2 / Xm - abs(Is75)**2 * Xs) / abs(Ir75)**2

Lr = Xr / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Rr :: {:.4f}".format(Rr))
    print("Lr :: {:.4f}m".format(Lr * 1000))

Rr :: 0.4874
Lr :: 3.3914m


In [38]:
Pe75 = 3 * (abs(Ir75)**2 * Rr * ((1 - s75) / s75))
Br = (Pe75 - OUTPUT_POWER_75_PERCENT_LOAD) / (ROTOR_SPEED_75_PERCENT_LOAD * (math.pi / 30))**2

if VERBOSITY:
    print("Br :: {:.4f}".format(Br))

Br :: 0.0064


In [39]:
Vs100 = LINE_VOLTAGE_100_PERCENT_LOAD / math.sqrt(3)
Is100 = cmath.rect(INPUT_CURRENT_100_PERCENT_LOAD, -math.acos(POWER_FACTOR_100_PERCENT_LOAD))
S100 = Vs100 * Is100.conjugate()
Vm100 = Vs100 - Is100 * complex(Rs, Xs)
Im100 = Vm100 / (complex(0, Rm * Xm) / complex(Rm, Xm))
Ir100 = Is100 - Im100
Pe100 = 3 * ((Vm100 - Ir100 * complex(Rs, Xs)) * Ir100.conjugate()).real
s100 = (3 * abs(Ir100)**2 * Rr) / ((3 * abs(Ir100)**2 *Rr) + Pe100)
wr100 = (1 - s100) * SYNCHRONOUS_SPEED
Pmec100 = Pe100 - (Br * (wr100 * (math.pi / 30))**2)
n = (Pmec100 / (3 * S100.real))
if VERBOSITY:
    print("Ir[A] :: {:.4f}".format(abs(Ir100)))
    print("n[r/min] :: {:.4f}".format(wr100))
    print("P2[kW] :: {:.4f}".format(Pmec100 / 1000))
    print("n[%] :: {:.4f}".format(n * 100))

Ir[A] :: 12.6099
n[r/min] :: 1456.0664
P2[kW] :: 7.5574
n[%] :: 88.6952


In [40]:
with open("../reports/parameters.yaml", "w") as file:
    documents = yaml.dump({
            "provided_parameters": {
                "Vs": round((LINE_VOLTAGE_NO_LOAD / math.sqrt(3)) * math.sqrt(2), 6),
                "Jr": MOMENT_OF_INERTIA,
                "we": round((2 * math.pi) * FREQUENCY, 6),
                "P": NUMBER_OF_POLES,
            }, "estimated_parameters": {
                "Rs": round(Rs, 6),
                "Ls": round(Ls, 6),
                "Rm": round(Rm, 6),
                "Xm": round(Xm, 6),
                "Xr": round(Xr, 6),
                "Rr": round(Rr, 6),
                "Br": round(Br, 6),
                "Ir": round(abs(Ir100), 6),
            }
        }, 
        file
    )